# Merging Datasets

In [8]:
import pandas as pd
import os
import sys
from tqdm import tqdm
tqdm.pandas()
data_directory = "data/"


## Paired Datasets

In [12]:

print(os.path.abspath(data_directory))



aiSource = pd.read_pickle(data_directory + "ai-code/codenet-3.code.pkl")
aiSource2 = pd.read_pickle(data_directory + "ai-code/codenet-14b.code.pkl")

C:\Desktop 2\Repos\Code-Classifier\data


In [16]:
print(aiSource)
print(aiSource2)
combined = pd.concat([aiSource, aiSource2], ignore_index=True)
print(combined)

  identifier                                               code actual label
0     p04030  #include <stdio.h>\n#include <stdlib.h>\n\nint...           ai
1     p03242  char s[4];\nfgets(s, sizeof(s), stdin);\ns[3] ...           ai
2     p03001  int maxArea(int width, int height) {\n    retu...           ai
3     p02971  #include <stdio.h>\n#include <stdlib.h>\n\nint...           ai
4     p02407  #include <stdio.h>\n#include <string.h>\n\nint...           ai
5     p02400  #include <stdio.h>\n#include <math.h>\n\ndoubl...           ai
6     p02256  int a, b;\nscanf("%d %d", &a, &b);\n\nif (a < ...           ai
7     p00002  #include <stdio.h>\n#include <stdlib.h>\n\nint...           ai
  identifier                                               code actual label
0     p04030  #include <stdio.h>\n\nint main() {\n    char b...           ai
1     p03242  #include <stdio.h>\n\nint main() {\n    int n;...           ai
2     p03001  #include <stdio.h>\n#include <stdlib.h>\n#incl...           ai

In [ ]:
print(aiSource['identifier'])

In [ ]:
dataset_directory = "datasets/VedranLjubovic/chosen/"
humanCode = aiSource['identifier'].to_frame()
humanCode['code'] = [None for x in range(len(humanCode))]

def getCode(fileIdentifier):
  try:
    file = open(dataset_directory + fileIdentifier + ".c", "r")
    code = "\n".join(file.readlines())
    file.close()
  except Exception as e:
    sys.stderr.write("Error reading file: " + dataset_directory + fileIdentifier + ".c" + "\n")
    return None
  return code

humanCode['code'] = humanCode['identifier'].progress_apply(getCode)

print(humanCode)

In [81]:
human_df = humanCode['code'].copy()
ai_df = aiSource['code'].copy()

human_df.columns = ['code']
ai_df.columns = ['code']

human_df = human_df.to_frame()
ai_df = ai_df.to_frame()

human_df['actual label'] = "human"
ai_df['actual label'] = "llm"

merged_df = pd.concat([human_df, ai_df], ignore_index=True)

In [82]:
merged_df.to_pickle(data_directory + "prepared/" + "code.pkl")

# splitting data

In [2]:
embedding_directory = "data/prepared/run2/"
ratios = [80, 10, 10]

data = pd.read_pickle(embedding_directory + "embeddings.pkl")
data_num = len(data)

train_split = int(ratios[0]/sum(ratios)*data_num)
val_split = train_split + int(ratios[1]/sum(ratios)*data_num)

data = data.sample(frac=1, random_state=666)
train = data.iloc[:train_split]
dev = data.iloc[train_split:val_split]
test = data.iloc[val_split:]

print(train_split, val_split)

84 94


In [3]:
train.to_pickle(embedding_directory + "train.pkl")
dev.to_pickle(embedding_directory + "dev.pkl")
test.to_pickle(embedding_directory + "test.pkl")

In [11]:
print(train)

   actual label                                    code_embeddings
8         human  [-0.0055792592, 0.06428501, -0.036523633, -0.0...
57          llm  [-0.027133392, 0.009359762, -0.10520409, 0.011...
90          llm  [0.015876377, -0.053451587, -0.070226625, 0.01...
40        human  [-0.045699924, 0.02933702, -0.11228899, 0.0292...
67          llm  [0.008493862, 0.032250702, -0.08035441, -0.015...
..          ...                                                ...
11        human  [0.021025768, 0.010113711, -0.13329825, 0.0339...
20        human  [0.0009624716, 0.020897575, -0.10187468, 0.063...
86          llm  [-0.06238021, 0.041018948, -0.10947401, 0.0711...
13        human  [0.02036846, -0.012620583, -0.12502861, -0.029...
79          llm  [-0.060971193, 0.070538305, -0.16201116, -0.00...

[84 rows x 2 columns]


In [26]:
# filtered_data = train.loc[:, train.columns.str.startswith('code_')]
# filtered_data['actual label'] = train['actual label']
# filtered_data['code_embeddings'] = filtered_data['code_embeddings'].apply(lambda x: x.flatten())


data_expanded = pd.DataFrame(train['code_embeddings'].tolist(), index=train.index)  # Expands each ndarray into separate columns
filtered_data = pd.concat([data_expanded, train[['actual label']]], axis=1)
# print(train['actual label'])
print(filtered_data) 

           0         1         2         3         4         5         6  \
8  -0.005579  0.064285 -0.036524 -0.022370  0.007151 -0.011820  0.000123   
57 -0.027133  0.009360 -0.105204  0.011347 -0.132737  0.003184 -0.019521   
90  0.015876 -0.053452 -0.070227  0.016451 -0.106307 -0.039220 -0.111423   
40 -0.045700  0.029337 -0.112289  0.029299 -0.094496  0.048511 -0.039841   
67  0.008494  0.032251 -0.080354 -0.015844 -0.059486  0.064062 -0.086975   
..       ...       ...       ...       ...       ...       ...       ...   
11  0.021026  0.010114 -0.133298  0.033915 -0.011136 -0.005381 -0.018329   
20  0.000962  0.020898 -0.101875  0.063812 -0.064141 -0.003297  0.016214   
86 -0.062380  0.041019 -0.109474  0.071166 -0.063021  0.049664 -0.035394   
13  0.020368 -0.012621 -0.125029 -0.029516 -0.077637 -0.018517 -0.033507   
79 -0.060971  0.070538 -0.162011 -0.003407 -0.026897  0.116769 -0.018240   

           7         8         9  ...       247       248       249       250  \
8   0.

# Different Testing Dataset

In [ ]:
testDirectory = "datasets/VedranLjubovic/chosen/"
testFiles = os.listdir(testDirectory)
testFiles = [x[:-2] for x in testFiles if x.endswith(".c")]
print(testFiles)

[]


In [14]:
humanTestCode = pd.DataFrame()
humanTestCode['code'] = [x for x in testFiles if getCode(x) is not None]

def getCode(fileIdentifier):
  try:
    file = open(testDirectory + fileIdentifier, "r")
    code = "\n".join(file.readlines())
    file.close()
  except Exception as e:
    sys.stderr.write("Error reading file: " + testDirectory + fileIdentifier + ".c" + "\n")
    return None
  return code

humanTestCode['code'] = humanTestCode['code'].progress_apply(getCode)

print(humanTestCode)

Error reading file: datasets/VedranLjubovic/testChosen/Z3-Z2-4770.c.c
Error reading file: datasets/VedranLjubovic/testChosen/Z4-Z2-2160.c.c
Error reading file: datasets/VedranLjubovic/testChosen/Z4-Z4-2254.c.c
Error reading file: datasets/VedranLjubovic/testChosen/Z4-Z4-7125.c.c
Error reading file: datasets/VedranLjubovic/testChosen/Z5-Z1-7888.c.c
100%|███████████████████████████████████████████████████████| 175/175 [00:00<00:00, 8822.68it/s]

                                                  code
0    #include <stdio.h>\n\n\n\nint main() {\n\n\tdo...
1    #include <stdio.h>\n\n\n\nint main() {\n\n\tfl...
2    #include <stdio.h>\n\n\n\n#define I_parcijalni...
3    #include <stdio.h>\n\n#define eps 0.0001\n\nin...
4    #include <stdio.h>\n\n\n\nint main() {\n\n\tin...
..                                                 ...
170  #include <stdio.h>\n\n#include <string.h>\n\n#...
171  /*\n\n#include <stdio.h>\n\n#include <string.h...
172  #include <stdio.h>\n\n#include <string.h>\n\n#...
173  #include <stdio.h>\n\n#include <string.h>\n\n#...
174  #include <stdio.h>\n\n#include <string.h>\n\n#...

[175 rows x 1 columns]


In [15]:
humanTestCode['actual label'] = "human"

In [10]:
print(humanTestCode)

                                                  code actual label
0    #include <stdio.h>\n\n\n\nint main() {\n\n\tdo...        human
1    #include <stdio.h>\n\n\n\nint main() {\n\n\tfl...        human
2    #include <stdio.h>\n\n\n\n#define I_parcijalni...        human
3    #include <stdio.h>\n\n#define eps 0.0001\n\nin...        human
4    #include <stdio.h>\n\n\n\nint main() {\n\n\tin...        human
..                                                 ...          ...
175  #include <stdio.h>\n\n#include <string.h>\n\n#...        human
176  /*\n\n#include <stdio.h>\n\n#include <string.h...        human
177  #include <stdio.h>\n\n#include <string.h>\n\n#...        human
178  #include <stdio.h>\n\n#include <string.h>\n\n#...        human
179  #include <stdio.h>\n\n#include <string.h>\n\n#...        human

[180 rows x 2 columns]


In [16]:
humanTestCode.to_pickle(data_directory + "prepared/" + "testCode.pkl")